In [11]:
import degirum as dg
import cv2 as cv
import numpy as np

In [12]:
model_zoo=dg.connect_model_zoo('localhost')

In [19]:
face_model=model_zoo.load_model('yolo_v5s_face_det--512x512_quant_n2x_orca_1')
mask_model=model_zoo.load_model('mobilenet_v2_mask_yn_cls--224x224_float_n2x_orca_1')
face_model.image_backend='pil'
face_model.input_resize_method='bilinear'
face_model.input_numpy_colorspace='BGR'
mask_model.image_backend='pil'
mask_model.input_resize_method='bilinear'
mask_model.input_numpy_colorspace='BGR'
mask_model.overlay_font_scale=4
face_model.output_confidence_threshold=0.7

In [20]:
cap=cv.VideoCapture(0)

if not cap.isOpened():
    print("cannot open camera")
    exit()
f1=[]

def source(data):
    while True:
        if data:
            yield data.pop(0)
        else:
            ret, frame = cap.read()
            f1.append(frame)

In [21]:
for faces in person_model.predict_batch(source(f1)):
    #Use this to show face detection results
    cv.imshow('faces',np.asarray(faces.image_overlay)[:,:,::-1])
    for index,face in enumerate(faces.results):
        face_region=faces.image.crop(face['bbox'])
        mask_results=mask_model(face_region)
        #for every face it will show one more stream
        #probably need better ways to show the results
        cv.imshow('faces'+str(index),np.asarray(mask_results.image_overlay)[:,:,::-1])
    if cv.waitKey(1)==ord('q'):
        break
cap.release()
cv.destroyAllWindows()